In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data=pd.read_csv('train_v9rqX0R.csv')


In [ ]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [ ]:
data.shape

(8523, 12)

In [ ]:
data.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [ ]:
num=data.select_dtypes(include=np.number)
cat=data.select_dtypes(exclude=np.number)

In [ ]:
num.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [ ]:
cat.describe(include=object)

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,8523,8523,8523,8523,6113,8523,8523
unique,1559,5,16,10,3,3,4
top,FDG33,Low Fat,Fruits and Vegetables,OUT027,Medium,Tier 3,Supermarket Type1
freq,10,5089,1232,935,2793,3350,5577


In [ ]:
for i in cat.columns:
    print(i,data[i].nunique(),data[i].unique(),sep='\n')
    print('----------------------------------------')

Item_Identifier
1559
['FDA15' 'DRC01' 'FDN15' ... 'NCF55' 'NCW30' 'NCW05']
----------------------------------------
Item_Fat_Content
5
['Low Fat' 'Regular' 'low fat' 'LF' 'reg']
----------------------------------------
Item_Type
16
['Dairy' 'Soft Drinks' 'Meat' 'Fruits and Vegetables' 'Household'
 'Baking Goods' 'Snack Foods' 'Frozen Foods' 'Breakfast'
 'Health and Hygiene' 'Hard Drinks' 'Canned' 'Breads' 'Starchy Foods'
 'Others' 'Seafood']
----------------------------------------
Outlet_Identifier
10
['OUT049' 'OUT018' 'OUT010' 'OUT013' 'OUT027' 'OUT045' 'OUT017' 'OUT046'
 'OUT035' 'OUT019']
----------------------------------------
Outlet_Size
3
['Medium' nan 'High' 'Small']
----------------------------------------
Outlet_Location_Type
3
['Tier 1' 'Tier 3' 'Tier 2']
----------------------------------------
Outlet_Type
4
['Supermarket Type1' 'Supermarket Type2' 'Grocery Store'
 'Supermarket Type3']
----------------------------------------


### We can see that we have missing vlue in two columns ,also minimum value for item_visibility is zero which does not seem reasonable,Item identifier have too many unique values ao it seems it will not be very useful in predicting ,Item_Fat_Content also have multiple entry for same type so we need to update the column,

#### Item_fat_Content

In [ ]:
data['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [ ]:
data['Item_Fat_Content']=data['Item_Fat_Content'].replace({'LF':'Low Fat','reg':'Regular','low fat':'Low Fat'})

In [ ]:
data['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

#### Outlet_size

In [ ]:
data['Outlet_Size'].isnull().sum()

2410

In [ ]:
data[['Outlet_Identifier','Outlet_Size']].groupby(['Outlet_Identifier']).agg('count')

,Outlet_Size
Outlet_Identifier,
OUT010,0
OUT013,932
OUT017,0
OUT018,928
OUT019,528
OUT027,935
OUT035,930
OUT045,0
OUT046,930


In [ ]:
#Import mode function:
from scipy.stats import mode

#Determing the mode for each
outlet_size_mode = data.pivot_table(values='Outlet_Size', columns='Outlet_Type',aggfunc=(lambda x:mode(x).mode[0]) )
print ('Mode for each Outlet_Type:')
print (outlet_size_mode)

#Get a boolean variable specifying missing Item_Weight values
miss_bool = data['Outlet_Size'].isnull() 

#Impute data and check #missing values before and after imputation to confirm
print ('\nOrignal #missing: %d'% sum(miss_bool))
data.loc[miss_bool,'Outlet_Size'] = data.loc[miss_bool,'Outlet_Type'].apply(lambda x: outlet_size_mode[x])
print (sum(data['Outlet_Size'].isnull()))

Mode for each Outlet_Type:
Outlet_Type Grocery Store Supermarket Type1 Supermarket Type2 Supermarket Type3
Outlet_Size         Small             Small            Medium            Medium

Orignal #missing: 2410
0


#### Item weight

In [ ]:
data['Item_Weight'] = data['Item_Weight'].fillna(data['Item_Weight'].dropna().mean())

#### item_visibility

In [ ]:
data['Item_Visibility']=data['Item_Visibility'].replace(0,data['Item_Visibility'].mean())

In [ ]:
data.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [ ]:
numf=data.select_dtypes(include=np.number)
catf=data.select_dtypes(exclude=np.number)

In [ ]:
catf.columns

Index(['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')

In [ ]:
catf_o=data[['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type']]
catf_oe=pd.get_dummies(catf_o)

In [ ]:
catf_oe.head()

,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Size_High,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,1,0,1,0,0,0,1,0,0,1,0,0
1,0,1,0,0,1,0,1,0,0,0,1,0
2,1,0,1,0,0,0,1,0,0,1,0,0
3,0,1,0,0,1,0,0,1,1,0,0,0
4,1,0,0,0,1,1,0,0,0,1,0,0


In [ ]:
catf_l=data[['Item_Identifier', 'Item_Type', 'Outlet_Identifier']]
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
catf_le=catf_l.apply(lb.fit_transform)

In [ ]:
catf_le.head()

,Item_Identifier,Item_Type,Outlet_Identifier
0,156,4,9
1,8,14,3
2,662,10,9
3,1121,6,0
4,1297,9,1


In [ ]:
numf.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
0,9.30,0.016047,249.8092,1999,3735.1380
1,5.92,0.019278,48.2692,2009,443.4228
2,17.50,0.016760,141.6180,1999,2097.2700
3,19.20,0.066132,182.0950,1998,732.3800
4,8.93,0.066132,53.8614,1987,994.7052


In [ ]:
numf.skew()

Item_Weight                  0.090561
Item_Visibility              1.286877
Item_MRP                     0.127202
Outlet_Establishment_Year   -0.396641
Item_Outlet_Sales            1.177531
dtype: float64

In [ ]:
dataf=pd.concat((numf,catf_le,catf_oe),axis=1)

In [ ]:
#split train into target and input
y=dataf['Item_Outlet_Sales']
x=dataf.drop('Item_Outlet_Sales',axis=1)

In [ ]:
from statsmodels.api import OLS
import statsmodels.api as sm
model1=OLS(y,sm.add_constant(x)).fit()
model1.summary()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Item_Outlet_Sales   R-squared:                       0.563
Model:                            OLS   Adj. R-squared:                  0.563
Method:                 Least Squares   F-statistic:                     732.0
Date:                Fri, 20 Nov 2020   Prob (F-statistic):               0.00
Time:                        04:54:17   Log-Likelihood:                -71991.
No. Observations:                8523   AIC:                         1.440e+05
Df Residuals:                    8507   BIC:                         1.441e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const                         -1.469e+05   3.95e+04     -3.720      0.000   -2.24e+05   -6.95e+04
Item_Weight                      -0.6381      2.899     -0.220      0.826      -6.320       5.044
Item_Visibility                -222.8213    263.650     -0.845      0.398    -739.640     293.997
Item_MRP                         15.5601      0.197     79.176      0.000      15.175      15.945
Outlet_Establishment_Year       178.4898     47.982      3.720      0.000      84.433     272.547
Item_Identifier                   0.0068      0.027      0.246      0.805      -0.047       0.061
Item_Type                        -0.7566      2.938     -0.257      0.797      -6.517       5.004
Outlet_Identifier               144.1578     45.645      3.158      0.002      54.682     233.634
Item_Fat_Content_Low Fat      -7.349e+04   1.98e+04     -3.721      0.000   -1.12e+05   -3.48e+04
Item_Fat_Content_Regular      -7.344e+04   1.98e+04     -3.718      0.000   -1.12e+05   -3.47e+04
Outlet_Location_Type_Tier 1   -4.812e+04   1.29e+04     -3.717      0.000   -7.35e+04   -2.27e+04
Outlet_Location_Type_Tier 2   -4.893e+04   1.32e+04     -3.717      0.000   -7.47e+04   -2.31e+04
Outlet_Location_Type_Tier 3   -4.988e+04   1.34e+04     -3.725      0.000   -7.61e+04   -2.36e+04
Outlet_Size_High              -4.579e+04   1.23e+04     -3.724      0.000   -6.99e+04   -2.17e+04
Outlet_Size_Medium            -5.078e+04   1.37e+04     -3.714      0.000   -7.76e+04    -2.4e+04
Outlet_Size_Small             -5.037e+04   1.35e+04     -3.722      0.000   -7.69e+04   -2.38e+04
Outlet_Type_Grocery Store     -3.778e+04   9692.121     -3.898      0.000   -5.68e+04   -1.88e+04
Outlet_Type_Supermarket Type1 -3.861e+04   1.04e+04     -3.696      0.000   -5.91e+04   -1.81e+04
Outlet_Type_Supermarket Type2 -3.814e+04   1.02e+04     -3.734      0.000   -5.82e+04   -1.81e+04
Outlet_Type_Supermarket Type3 -3.241e+04   9150.652     -3.542      0.000   -5.04e+04   -1.45e+04
==============================================================================
Omnibus:                      966.079   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2309.093
Skew:                           0.670   Prob(JB):                         0.00
Kurtosis:                       5.170   Cond. No.                     2.75e+19
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.22e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 
vif=pd.DataFrame()
vif['VIF']=[variance_inflation_factor(x.values,i) for i in range(x.shape[1])]
vif['column']=x.columns
vif.sort_values(by='VIF',ascending=False)            

/usr/local/lib/python3.6/dist-packages/statsmodels/stats/outliers_influence.py:185: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,VIF,column
9,inf,Outlet_Location_Type_Tier 1
10,inf,Outlet_Location_Type_Tier 2
17,inf,Outlet_Type_Supermarket Type2
16,inf,Outlet_Type_Supermarket Type1
15,inf,Outlet_Type_Grocery Store
14,inf,Outlet_Size_Small
13,inf,Outlet_Size_Medium
12,inf,Outlet_Size_High
11,inf,Outlet_Location_Type_Tier 3
18,inf,Outlet_Type_Supermarket Type3


In [ ]:
inp1=x.drop(['Outlet_Location_Type_Tier 1','Outlet_Size_High','Outlet_Type_Supermarket Type1',
             'Outlet_Establishment_Year','Outlet_Size_Small','Outlet_Identifier','Item_Fat_Content_Low Fat'
            ],axis=1)
vif=pd.DataFrame()
vif['VIF']=[variance_inflation_factor(inp1.values,i) for i in range(inp1.shape[1])]
vif['column']=inp1.columns
vif.sort_values(by='VIF',ascending=False)

,VIF,column
0,7.429546,Item_Weight
2,5.233895,Item_MRP
8,4.844644,Outlet_Size_Medium
7,4.116967,Outlet_Location_Type_Tier 3
3,3.649963,Item_Identifier
4,3.629268,Item_Type
11,3.157188,Outlet_Type_Supermarket Type3
10,3.136356,Outlet_Type_Supermarket Type2
1,3.129738,Item_Visibility
6,2.823489,Outlet_Location_Type_Tier 2


In [ ]:
#Create ols model after removing multicollinierity
model2=OLS(y,sm.add_constant(inp1)).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Item_Outlet_Sales   R-squared:                       0.562
Model:                            OLS   Adj. R-squared:                  0.562
Method:                 Least Squares   F-statistic:                     911.7
Date:                Fri, 20 Nov 2020   Prob (F-statistic):               0.00
Time:                        04:54:18   Log-Likelihood:                -72000.
No. Observations:                8523   AIC:                         1.440e+05
Df Residuals:                    8510   BIC:                         1.441e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const                            78.2846     67.545      1.159      0.246     -54.120     210.689
Item_Weight                      -0.4908      2.901     -0.169      0.866      -6.178       5.196
Item_Visibility                -215.7704    263.839     -0.818      0.413    -732.960     301.419
Item_MRP                         15.5627      0.197     79.124      0.000      15.177      15.948
Item_Identifier                   0.0062      0.027      0.225      0.822      -0.048       0.060
Item_Type                        -0.7162      2.941     -0.244      0.808      -6.482       5.049
Item_Fat_Content_Regular         51.5704     26.073      1.978      0.048       0.461     102.680
Outlet_Location_Type_Tier 2      51.4435     39.728      1.295      0.195     -26.433     129.320
Outlet_Location_Type_Tier 3      13.1319     41.644      0.315      0.753     -68.501      94.765
Outlet_Size_Medium               88.9028     49.924      1.781      0.075      -8.960     186.766
Outlet_Type_Grocery Store     -1916.6014     44.713    -42.865      0.000   -2004.250   -1828.953
Outlet_Type_Supermarket Type2  -387.2281     66.950     -5.784      0.000    -518.466    -255.990
Outlet_Type_Supermarket Type3  1340.0287     66.866     20.040      0.000    1208.955    1471.103
==============================================================================
Omnibus:                      963.090   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2297.638
Skew:                           0.668   Prob(JB):                         0.00
Kurtosis:                       5.164   Cond. No.                     1.97e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.97e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
#train test split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(inp1,y,test_size=0.3,random_state=10)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,KFold,cross_val_score
from sklearn.metrics import mean_squared_error,r2_score

In [ ]:
rf=RandomForestRegressor()
model_rf=rf.fit(x_train,y_train)

In [ ]:
ypred_rf_train=model_rf.predict(x_train)
ypred_rf_test=model_rf.predict(x_test)

#Rmse 
rmse_train=np.sqrt(mean_squared_error(y_train,ypred_rf_train))
rmse_test=np.sqrt(mean_squared_error(y_test,ypred_rf_test))
                  
#r2 squared
r2_train=r2_score(y_train,ypred_rf_train)
r2_test=r2_score(y_test,ypred_rf_test)
                  
print(f"RMSE for train - :{rmse_train}")
print(f"RMSE for test - : {rmse_test}")  
                  
                  
print(f"R2 score for train - :{r2_train}") 
print(f"R2 score for test - :{r2_test}")  

RMSE for train - :431.57709333579896
RMSE for test - : 1175.0289107467597
R2 score for train - :0.9360073139503748
R2 score for test - :0.526251183705589


In [ ]:
#grid search for reducing 
rf=RandomForestRegressor()
param={ 'max_depth':[4,5,6],
       'max_features':['auto', 'sqrt', 'log2'],
       'warm_start':[True,False],
     'bootstrap':[True,False]}

search=GridSearchCV(rf,param_grid=param,cv=4)
search.fit(inp1,y)

GridSearchCV(cv=4, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [ ]:
search.best_params_

{'bootstrap': True,
 'max_depth': 6,
 'max_features': 'auto',
 'n_estimators': 100,
 'warm_start': True}

fitting the tunned parameter

In [ ]:
rf_gs=RandomForestRegressor(**search.best_params_,random_state=10).fit(x_train,y_train)

In [ ]:
ypred_rfgs_train=rf_gs.predict(x_train)
ypred_rfgs_test=rf_gs.predict(x_test)

#Rmse 
rmse_train_gs=np.sqrt(mean_squared_error(y_train,ypred_rfgs_train))
rmse_test_gs=np.sqrt(mean_squared_error(y_test,ypred_rfgs_test))
                  
#r2 squared
r2_train_gs=r2_score(y_train,ypred_rfgs_train)
r2_test_gs=r2_score(y_test,ypred_rfgs_test)
                  
print(f"RMSE for train - :{rmse_train_gs}")
print(f"RMSE for test - : {rmse_test_gs}")  
                  
                  
print(f"R2 score for train - :{r2_train_gs}") 
print(f"R2 score for test - :{r2_test_gs}")  

RMSE for train - :1040.6730040968416
RMSE for test - : 1092.746852436372
R2 score for train - :0.6279150142882226
R2 score for test - :0.5902771718684632


In [ ]:
rf_gs.fit(inp1,y)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_forest.py:359: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=6, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=10, verbose=0, warm_start=True)

In [ ]:
ypred_all=rf_gs.predict(inp1)

In [ ]:
ypred_all.min()

66.98490054552502

In [ ]:
test=pd.read_csv('test_AbJTz2l.csv')

In [ ]:
test.isnull().sum()

Item_Identifier                 0
Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [ ]:
test['Item_Fat_Content']=test['Item_Fat_Content'].replace({'LF':'Low Fat','reg':'Regular','low fat':'Low Fat'})

In [ ]:
#Import mode function:
from scipy.stats import mode

#Determing the mode for each
outlet_size_mode = test.pivot_table(values='Outlet_Size', columns='Outlet_Type',aggfunc=(lambda x:mode(x).mode[0]) )
print ('Mode for each Outlet_Type:')
print (outlet_size_mode)

#Get a boolean variable specifying missing Item_Weight values
miss_bool = test['Outlet_Size'].isnull() 

#Impute test and check #missing values before and after imputation to confirm
print ('\nOrignal #missing: %d'% sum(miss_bool))
test.loc[miss_bool,'Outlet_Size'] = test.loc[miss_bool,'Outlet_Type'].apply(lambda x: outlet_size_mode[x])
print (sum(test['Outlet_Size'].isnull()))

Mode for each Outlet_Type:
Outlet_Type Grocery Store Supermarket Type1 Supermarket Type2 Supermarket Type3
Outlet_Size         Small             Small            Medium            Medium

Orignal #missing: 1606
0


In [ ]:
test['Item_Weight'] = test['Item_Weight'].fillna(test['Item_Weight'].dropna().mean())

In [ ]:
test['Item_Visibility']=test['Item_Visibility'].replace(0,test['Item_Visibility'].mean())

In [ ]:
numf_t=test.select_dtypes(include=np.number)
catf_t=test.select_dtypes(exclude=np.number)

In [ ]:
catf_ot=test[['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type']]
catf_oet=pd.get_dummies(catf_ot)

In [ ]:
catf_lt=test[['Item_Identifier', 'Item_Type', 'Outlet_Identifier']]
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
catf_let=catf_lt.apply(lb.fit_transform)

In [ ]:
testf=pd.concat((numf_t,catf_let,catf_oet),axis=1)

In [ ]:
inp_test=testf[inp1.columns.to_list()]

In [ ]:
ytest_pred=rf_gs.predict(inp_test)

In [ ]:
ytest_pred.shape

(5681,)

In [ ]:
test['Item_Outlet_Sales']=ytest_pred

In [ ]:
test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDW58,20.750000,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1,1635.772846
1,FDW14,8.300000,Regular,0.038428,Dairy,87.3198,OUT017,2007,Small,Tier 2,Supermarket Type1,1321.106630
2,NCN55,14.600000,Low Fat,0.099575,Others,241.7538,OUT010,1998,Small,Tier 3,Grocery Store,590.601089
3,FDQ58,7.315000,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,Small,Tier 2,Supermarket Type1,2462.298813
4,FDY38,12.695633,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3,5627.429650


In [ ]:
result=test[['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales']]

In [ ]:
result.head()

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1635.772846
1,FDW14,OUT017,1321.106630
2,NCN55,OUT010,590.601089
3,FDQ58,OUT017,2462.298813
4,FDY38,OUT027,5627.429650


In [ ]:
result.min()

Item_Identifier        DRA12
Outlet_Identifier     OUT010
Item_Outlet_Sales    71.5652
dtype: object

In [ ]:
result.to_csv('result_bigmart_rf.csv')